In [1]:
import torch
import pandas as pd
import torch.optim as optim
import requests
from torch.nn import Parameter, Sigmoid
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.w1 = Parameter(torch.ones(24) + 0.01 * torch.randn(24))
        self.w2 = Parameter(torch.ones(24) + 0.01 * torch.randn(24))

    def forward(self, oracle_demand, oracle_solar, oracle_price):
        bid_demand = (oracle_demand - oracle_solar) * self.w1
        bid_price = oracle_price * self.w2
        bid_price.data.clamp_(max=7)
        return bid_demand, bid_price

In [3]:
demand_train_oracle = pd.read_csv('./Data/Demand_Train_pred.csv', header=None)
demand_train_actual = pd.read_csv('./Data/Demand_Train.csv', header=None)
solar_train_oracle = pd.read_csv('./Data/Solar_Train_pred.csv', header=None)
solar_train_actual = pd.read_csv('./Data/Solar_Train.csv', header=None)
price_train_oracle = pd.read_csv('./Data/Price_Train_pred.csv', header=None)
price_train_actual = pd.read_csv('./Data/Price_Train.csv', header=None)

In [4]:
def cost(bid_quantities, bid_prices, demands, solar_outputs, prices):
    BATTERY_CAPACITY = 25
    BATTERY_CAPACITY_PER_HOUR = 5
    BATTERY_EFFICIENCY = 0.8
    DISCOM_RATE = 7
    battery = Variable(torch.Tensor([0]))
    cost = Variable(torch.Tensor([0]))
    result = []
    for bid_quantity, bid_price, demand, solar_output, price in zip(bid_quantities.contiguous().view(-1), bid_prices.contiguous().view(-1), demands.contiguous().view(-1), solar_outputs.contiguous().view(-1), prices.contiguous().view(-1)):
        energy_from_sun = torch.min(solar_output, demand)
        sigmoid = Sigmoid()
        bid_won = sigmoid(1000 * (bid_price - price) / bid_price)
        energy_from_market = bid_won * bid_quantity
        energy_from_battery = torch.min(torch.clamp((demand - energy_from_sun - energy_from_market) / BATTERY_EFFICIENCY, min=0),
                                        torch.clamp(battery, max=BATTERY_CAPACITY_PER_HOUR)) * BATTERY_EFFICIENCY
        energy_from_DISCOM = torch.clamp(demand - energy_from_sun - energy_from_market - energy_from_battery, min=0)
        
        residual_energy = torch.clamp(energy_from_market + energy_from_sun - demand, min=0)
        battery = battery + torch.clamp(residual_energy, max=BATTERY_CAPACITY_PER_HOUR) - energy_from_battery / BATTERY_EFFICIENCY
        battery = torch.clamp(battery, min=0, max=BATTERY_CAPACITY)
        
        cost += energy_from_market * bid_price + energy_from_DISCOM * DISCOM_RATE
    return cost

In [5]:
data_size = demand_train_actual.values.shape[0]
epochs = 20
batch_size = 1
learning_rate = (10 ** (-8))
net = Net()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

In [6]:
for epoch in range(epochs):
    start = 0
    end = batch_size
    epoch_loss = 0
    while end <= data_size:
        net.zero_grad()
        bid_quantities, bid_prices = net(Variable(torch.Tensor(demand_train_oracle.values[start:end])), 
                              Variable(torch.Tensor(solar_train_oracle.values[start:end])), 
                              Variable(torch.Tensor(price_train_oracle.values[start:end])))
        loss = cost(bid_quantities, bid_prices,
             Variable(torch.Tensor(demand_train_actual.values[start:end])),
             Variable(torch.Tensor(solar_train_actual.values[start:end])),
             Variable(torch.Tensor(price_train_actual.values[start:end])))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.data[0]
        start += batch_size
        end += batch_size
    print('Epoch {}: {}'.format(epoch + 1, epoch_loss * 50 / data_size))
    params = list(net.parameters())
    w1 = params[0].data
    w2 = params[1].data
    print("w1: {}\nw2: {},".format(w1.numpy(), w2.numpy()))

Epoch 1: 407033.93082682294
w1: [ 1.01469612  1.00643194  1.00215697  0.98892272  1.00785398  1.01744711
  0.99934787  0.98949575  1.00426936  0.9945994   0.9834621   1.01097667
  0.99682701  1.00402474  0.99282575  0.99970084  0.99473643  1.00636637
  0.99729514  1.00394571  1.0045625   0.99529386  1.00218463  1.00311077]
w2: [ 1.05292606  1.06592309  1.02735984  1.06770992  1.02431941  1.02361393
  1.05371296  1.01993024  1.0086031   1.04684484  1.0275346   1.03232789
  1.05922353  1.05890596  1.04649043  1.04328859  1.01194823  1.07653844
  1.04921699  1.1263814   1.12482429  1.07612121  1.14975584  1.06964982],
Epoch 2: 365419.5177680121
w1: [ 1.02150178  1.01387513  1.00841284  0.99659175  1.01333773  1.02150095
  1.00225496  0.9875052   1.000103    0.99364889  0.98374283  1.01084542
  0.99780786  1.00614798  0.99471533  1.00103426  0.99664593  1.00564945
  0.99557877  1.0027988   0.99965829  0.99407715  1.0057081   1.00620294]
w2: [ 1.07881534  1.09826541  1.05332923  1.10469067 

Epoch 15: 318752.17405870225
w1: [ 1.08716142  1.08822548  1.08358014  1.0724833   1.07211626  1.06444061
  1.00787938  0.94300413  0.94315583  0.96069551  0.95876265  0.98416448
  0.9864887   0.99978268  0.98999912  0.99284279  0.99271297  0.98014408
  0.96799982  0.98662961  0.97984093  0.98619348  1.01168716  1.02225804]
w2: [ 1.18186104  1.18211818  1.17979956  1.17646968  1.18023837  1.17613316
  1.17635119  1.15825582  1.16289401  1.18089342  1.17663479  1.17405117
  1.17867434  1.18447542  1.17858934  1.17695189  1.17960978  1.17330492
  1.17447841  1.19124782  1.16664362  1.17552304  1.17914081  1.1765058 ],
Epoch 16: 318652.3972507053
w1: [ 1.08982456  1.09181583  1.08787632  1.07651019  1.07525694  1.06686401
  1.00751889  0.94181812  0.94089168  0.95890546  0.95764697  0.98319238
  0.98563999  0.99948853  0.98992884  0.99236733  0.99249953  0.97974151
  0.96772581  0.9865132   0.97983474  0.98618764  1.01175058  1.02268517]
w2: [ 1.18365133  1.18225944  1.17975426  1.1764701

In [7]:
demand_pub_oracle = pd.read_csv('./Data/Demand_LB_pred.csv', header=None)
solar_pub_oracle = pd.read_csv('./Data/Solar_LB_pred.csv', header=None)
price_pub_oracle = pd.read_csv('./Data/Price_LB_pred.csv', header=None)

In [8]:
def submit(name):
    url = 'http://interiit.tech:3000/upload'
    files = {'file': (name, open(name, 'rb'), '.text/csv', {'Content-Disposition': 'form-data'})}
    headers = {'Host': 'interiit.tech:3000', 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0',\
               'Accept': 'application/json, text/plain, */*', 'Accept-Language': 'en-US,en;q=0.5', 'Accept-Encoding': 'gzip, deflate',\
               'Referer': 'http://interiit.tech:3000/', 'Origin':'http://interiit.tech:3000'}
    r = requests.post(url, files=files, headers=headers)
    
def score(team_id):
    return pd.read_csv('http://interiit.tech:3000/results.csv').loc[team_id]['Score']

In [9]:
bid_quantity = ((demand_pub_oracle - solar_pub_oracle).values * net.state_dict()['w1'].numpy()).flatten()
bid_price = (price_pub_oracle.values * net.state_dict()['w2'].numpy()).flatten()
submission = pd.DataFrame([bid_price, bid_quantity])
submission.T.to_csv('23.csv', header=None, index=None)
submit('23.csv')
score(23)

319579